In [1]:
import numpy as np
from steps import *
from tqdm import tqdm
from optimal import *
import matplotlib.pyplot as plt
import optuna
import pickle
import warnings
warnings.filterwarnings("error")


In [2]:
def plot_res(x_axis, results, x_label, y_label, label, title, path):
    plt.grid()
    plt.plot(x_axis, results.mean(axis=1), label=label)
    plt.fill_between(x_axis, results.mean(axis=1)+results.std(axis=1), results.mean(axis=1)-results.std(axis=1),
                     color='r', alpha=0.5)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend()
    plt.savefig(path+".png")
    plt.clf()

    plt.grid()
    plt.boxplot(results.T, positions=x_axis)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend()
    plt.savefig("BP-"+path+".png")
    plt.clf()

In [3]:
d = 20
opt_r = 5
m = 20
T_r = 100
times = 10
opt_sigma = 3
sigma_type = matrix_type.DIAG
s_type = matrix_type.IDENTITY

name = f"optuna_optimization_d={d}_r={opt_r}_sigma={sigma_type}_s={s_type}_m={m}_Tr={T_r}_var={opt_sigma}"
study = optuna.create_study(study_name=f'{name}',
                            storage=f'sqlite:///./../optuna/{name}.db ',
                            direction='minimize', load_if_exists=True)

[I 2023-05-19 13:23:54,600] Using an existing study with name 'optuna_optimization_d=20_r=5_sigma=matrix_type.DIAG_s=matrix_type.IDENTITY_m=20_Tr=100_var=3' instead of creating a new one.


In [9]:
# Sigma
sigma_max = 3.0
sigma_min = 0.5
spaces = 0.5
load = False

suffix = f'{sigma_min}-{sigma_max}-{spaces}_opt-r={opt_r}_opt-sigma={opt_sigma}_Tr={1000}'

params = study.best_params

if not load:
    d_sigmas = np.linspace(sigma_min, sigma_max, int((sigma_max-sigma_min)/spaces)+1)
    results = np.zeros([len(d_sigmas), times])
    for idx, d_sigma in enumerate(d_sigmas):
        print(f" ***** Sigma={d_sigma} *****")
        regret_list, regret_mix_list = run_algorithm(d=d, r=opt_r, m=m, sigma_type=sigma_type, s_type=s_type,
                                                     times=times, d_sigma=d_sigma, T_r=T_r*10,
                                                     **params)
        ratio = regret_list.min(axis=1) / regret_mix_list
        results[idx, :] = ratio

    pickle.dump(results, open(f'acc-ratio_vs_sigma0_{suffix}.p', 'wb'))
    pickle.dump(d_sigmas, open(f'sigma0-acc_{suffix}.p', 'wb'))

else:
    results = pickle.load(open(f'acc-ratio_vs_sigma0_{suffix}.p', 'rb'))
    d_sigmas = pickle.load(open(f'sigma0-acc_{suffix}.p', 'rb'))

plot_res(x_axis=d_sigmas, results=results,
         x_label=r'$\sigma_0$', y_label='Accuracy Ratio', label=f'd={d}, r={opt_r}',
         title=r'results accuracy ratio vs $\sigma_0$',
         path=f"results_accuracy_ratio_sigma0_{suffix}")

 ***** Sigma=0.5 *****
l_star=13 ; regret_mix=0.7146600483686888 ; last_regret=0.9871793891352602
l_star=14 ; regret_mix=0.7232103601409025 ; last_regret=0.9479001520327295
l_star=15 ; regret_mix=0.6922131642090811 ; last_regret=0.8892558575810114
l_star=13 ; regret_mix=0.7097215038741017 ; last_regret=0.9773314262571832
l_star=14 ; regret_mix=0.6927592262526151 ; last_regret=0.904878112138482
l_star=14 ; regret_mix=0.7066911790463005 ; last_regret=0.9195923660171511
l_star=16 ; regret_mix=0.7107677240426603 ; last_regret=0.9298097897453939
l_star=17 ; regret_mix=0.7101730093070882 ; last_regret=0.9183815002638523
l_star=14 ; regret_mix=0.6750315788296939 ; last_regret=1.0096366348433887
l_star=16 ; regret_mix=0.7177373484520194 ; last_regret=0.9659546626877928
 ***** Sigma=1.0 *****
l_star=8 ; regret_mix=0.6418736265661956 ; last_regret=0.9092611909709609
l_star=10 ; regret_mix=0.47742265927387995 ; last_regret=0.7034375462502582
l_star=11 ; regret_mix=0.48763540190535165 ; last_regre

Exception ignored in: <_io.FileIO name='acc-ratio_vs_sigma0_0.5-3.0-0.5_opt-r=5_opt-sigma=3_Tr=1000.p' mode='wb' closefd=True>
Traceback (most recent call last):
  File "C:\Users\neria\AppData\Local\Temp\ipykernel_19980\425125808.py", line 22, in <module>
Exception ignored in: <_io.FileIO name='sigma0-acc_0.5-3.0-0.5_opt-r=5_opt-sigma=3_Tr=1000.p' mode='wb' closefd=True>
Traceback (most recent call last):
  File "C:\Users\neria\AppData\Local\Temp\ipykernel_19980\425125808.py", line 23, in <module>
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


last_regret=0.8471097764155282


<Figure size 640x480 with 0 Axes>

In [6]:
study.best_params



{'avg_frac_r': 9,
 'beta_f': 0.776848551044915,
 'beta_r': 0.934083253446272,
 'lr_f': 0.6221613189836414,
 'lr_r': 0.8334222520229866,
 'stop_frac_r': 9}

In [ ]:
# r
r_max = 10
r_min = 1
spaces = 1
load = False

suffix = f'{r_min}-{r_max}-{spaces}_opt-r={opt_r}_opt-sigma={opt_sigma}'

if not load:
    r_list = range(r_min, r_max, spaces)
    results = np.zeros([len(r_list), times])
    for idx, r in enumerate(r_list):
        regret_list, regret_mix_list = run_algorithm(d=d, r=r, m=m, sigma_type=sigma_type, s_type=s_type,
                                                     times=times, d_sigma=opt_sigma, T_r=T_r,
                                                     **study.best_params)
        ratio = regret_list.min(axis=1) / regret_mix_list
        results[idx, :] = ratio

    pickle.dump(results, open(f'acc-ratio_vs_r_.p', 'wb'))
    pickle.dump(r_list, open(f'r-acc_{suffix}.p', 'wb'))

else:
    results = pickle.load(open(f'acc-ratio_vs_r_{suffix}.p', 'rb'))
    r_list = pickle.load(open(f'r-acc_{suffix}.p', 'rb'))

plot_res(x_axis=r_list, results=results,
         x_label='r', y_label='Accuracy Ratio', label=f"d={d}"+r" $\sigma_0$="+f"{opt_sigma}",
         title='results accuracy ratio vs r',
         path=f"results_accuracy_ratio_r_{suffix}")


In [ ]:
# d
load = False
d_max = 20
d_min = opt_r+2
spaces = 1
suffix = f'{d_min}-{d_max}-{spaces}_opt-r={opt_r}_opt-sigma={opt_sigma}'

if not load:
    d_list = range(d_min, d_max, spaces)
    results = np.zeros([len(d_list), times])

    for idx, d in enumerate(d_list):
        regret_list, regret_mix_list = run_algorithm(d=d, r=opt_r, m=m, sigma_type=sigma_type, s_type=s_type,
                                                     times=times, d_sigma=opt_sigma, T_r=T_r,
                                                     **study.best_params)
        ratio = regret_list.min(axis=1) / regret_mix_list
        results[idx, :] = ratio

    pickle.dump(results, open(f'acc-ratio_vs_d_{suffix}.p', 'wb'))
    pickle.dump(d_list, open(f'd-acc_{suffix}.p', 'wb'))

else:
    results = pickle.load(open(f'acc-ratio_vs_d_{suffix}.p', 'rb'))
    d_list = pickle.load(open(f'd-acc_{suffix}.p', 'rb'))

plot_res(x_axis=d_list, results=results,
         x_label='d', y_label='Accuracy Ratio', label=f"r={opt_r}"+r" $\sigma_0$="+f"{opt_sigma}",
         title='results accuracy ratio vs d',
         path=f"results_accuracy_ratio_d_{suffix}")

In [ ]:
# Regrets
load = False

if not load:
    d=20
    regret_list, regret_mix_list = run_algorithm(d=d, r=opt_r, m=m, sigma_type=sigma_type, s_type=s_type,
                                                     times=times, d_sigma=opt_sigma, T_r=T_r,
                                                     **study.best_params)
    pickle.dump(regret_list, open(f'regrets.p', 'wb'))

else:
    regret_list = pickle.load(open(f'regrets.p', 'rb'))


plot_res(x_axis=range(m+2), results=regret_list.T,
         x_label='#Iteration', y_label='Regret', label=f'd={d} r={opt_r} '+r'$\sigma_0$='+f'{opt_sigma}',
         title='Regret vs #Iteration',
         path=f"Regret_vs_Iteration_r={opt_r}_var={opt_sigma}")
